In [3]:
# !pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
browser_path = 'D:/Desktop/chromedriver.exe'
# setting the chromedriver path and initializing driver
driver = webdriver.Chrome(executable_path=browser_path)
# create master df to append to
master_df = pd.DataFrame()

In [32]:
from time import sleep
import random
def sleep_for(opt1, opt2):
    time_for = random.uniform(opt1, opt2)
    time_for_int = int(round(time_for))
    sleep(abs(time_for_int - time_for))
    for i in range(time_for_int, 0, -1):
        sleep(1)

In [33]:
# !pip install progressbar

In [34]:
from progressbar import ProgressBar
pbar = ProgressBar()
urls = ['https://twitter.com/search?q=simpsons%20(predicted%20OR%20covid)&src=typed_query']
# how many times should the browser scroll down
scroll_down_num = 4  
# the element we are obtaining from the webpage
post_element_xpath = '//div/div/article/div/div'
# loop through your list of urls
for url in pbar(urls):
    driver.get(url)
    sleep_for(10, 15)  # sleep a while
    # scroll x number of times
    for i in range(0, scroll_down_num):
        # scroll down
        driver.find_element_by_xpath('//body').send_keys(Keys.END)
        sleep_for(4, 7)
    # get a list of tweets
    post_list = driver.find_elements_by_xpath(post_element_xpath)
    # get the text only from each element
    post_text = [x.text for x in post_list]
    # create temporary dataset of each tweet
    temp_df = pd.DataFrame(post_text, columns={'all_text'})
    # append the temporary dataset to the dataset we will save
    master_df = master_df.append(temp_df) 

100% |########################################################################|


In [35]:
master_df

,all_text
0,Accidental Partridge\n@AccidentalP\n·\nNov 1\n...
1,DP\n@djcp87\n·\nNov 1\n“The Simpson’s Predicte...
2,002\n@baleriebear\n·\n19h\nquick everybody che...
3,Waluigi Galleani\n@Thedoomscrolls\n·\n18h\nthe...
4,Dan Povenmire\n@DanPovenmire\n·\nOct 31\nI cal...
5,Indie (COMMS OPEN)\n@burntcinnabun\n·\nOct 31\...
6,appaboy\n@appaboy\n·\nNov 3\ni cant believe si...
7,Snoc Makes Friends\n@SnocHog\n·\n18h\nthe simp...
8,Leezer Beam\n@pickleknee\n·\nNov 1\nWow. Simps...
9,BryBryBlessed\n@BryBryBlessed\n·\n18h\nReplyin...


In [36]:
def parse_text(text):
    # split by new line
    text_list = str.splitlines(text) 
    # get the username (always the first list element)
    username = text_list[0]  
    # within the first few elements, find the element
    # with the @ symbol, this will be the user handle
    handle = ''.join(x for x in text_list[1:3] if '@' in x)
    # get the date, using the single dot to identify its 
    # index location
    dot_position = text_list[1:4].index('·')  
    date = text_list[dot_position + 2]  # date comes after dot
    # check if its a reply to someone else
    if text_list[4] == "Replying to ":
        reply_to = True
        reply_to_handle = text_list[5]
        text = text_list[6]
    else:
        reply_to = False
        reply_to_handle = ''
        # find the longest string within list index 4:6
        # this will be the tweet text
        text = max(text_list[4:6], key=len)
    # return the variables we have parse from the text
    return pd.Series([username, handle,
                      date, reply_to, reply_to_handle, text])
# run the parse function via pandas apply
master_df[['username', 'handle', 'date', 'reply_to', 'reply_to_handle', 'all_text']] = master_df['all_text'].apply(parse_text)
# export csv
# df.to_csv('output.csv')
print(master_df)

                                             all_text  \
0                          “STOP GETTING BOND WRONG!”   
1                         “The Simpson’s Predicted it   
2   quick everybody check the simpsons episodes to...   
3            the Simpsons predicted this would happen   
4    I called it!  You are not going to believe this!   
5   There's gonna be another simpsons-like conspir...   
6   i cant believe simpsons predicted that humans ...   
7             the simpsons predicted deez nuts guy!!!   
8                         Wow. Simpsons predicted it.   
9            The simpsons predicted the future lowkey   
10                    The Simpsons predicted disposal   

                            username            handle    date  reply_to  \
0               Accidental Partridge      @AccidentalP   Nov 1     False   
1                                 DP           @djcp87   Nov 1     False   
2                                002      @baleriebear     19h     False   
3          